In [1]:
import numpy as np
from pychunkedgraph.graph import ChunkedGraph, ChunkedGraphMeta
from pychunkedgraph.graph.meta import ChunkedGraphMeta, GraphConfig, DataSource

ds = DataSource(
  EDGES= "gs://chunkedgraph/minnie65_0/edges",
  COMPONENTS= "gs://chunkedgraph/minnie65_0/components",
  WATERSHED= "gs://microns-seunglab/minnie65/ws_minnie65_0"
)
gc = GraphConfig(
  CHUNK_SIZE= [256, 256, 512]
)
cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

In [2]:
edges = cg.read_chunk_edges([cg.get_chunk_id(layer=2,x=50,y=100,z=0)])
edges = edges["between"].get_pairs()
nodes = np.unique(edges)
parents = cg.get_roots(nodes)
d = dict(zip(parents, nodes))
print(d)

{864691138338302132: 75582971913961693, 864691137629078617: 75512534450307707, 864691137995403800: 75582903194484819, 864691137995404568: 75582834475009044, 864691138741942725: 75582971913961542, 864691137995404824: 75582903194484963, 864691136334203586: 75582971913961503}


In [3]:
edges = np.array([[95372463227012287, 95372463227012312]])
print(cg.get_cross_chunk_edges_layer(edges))
print(cg.get_roots(edges[0]))

[1]
[864691139630589421 864691133787377379]


In [4]:
parents = np.uint64(edges[0][0])
while parents and parents.size:
    parents = cg.get_parent(parents)
    print(parents)

167430057264939027
239440097424965679
311402583707100284
383549238186954306
455215406085692589
600386125324760614
668784544669452312
864691139630589421
None


In [5]:
parents = np.uint64(edges[0][1])
while parents and parents.size:
    parents = cg.get_parent(parents)
    print(parents)

167430057264939029
864691133787377379
None


In [6]:
from pychunkedgraph.graph.edits import add_edges
from pychunkedgraph.graph.utils.context_managers import TimeIt

edges = np.array(edges, dtype=np.uint64)
with TimeIt("add_edge"):
    print(add_edges(cg, atomic_edges=edges))

Start add_edge
   Start get_parents edges
   4.220008850097656e-05
{167430057264939027: {2: array([[95372463227022388, 95302094482845948],
       [95372463227022388, 95302094482843792],
       [95372463227022383, 95302094482845950],
       [95372463227022383, 95302094482845948],
       [95372463227023310, 95302094482845948],
       [95372463227023310, 95302094482844793],
       [95372463227027495, 95302094482844793],
       [95372463227027495, 95302094482845948],
       [95372463227027495, 95302094482845950]], dtype=uint64)}, 167430057264939029: {}}
Cannot find root id 455215406085308417, 8, 2020-02-06 16:23:49.260000+00:00
Cannot find root id 600386125323829249, 12, 2020-02-06 16:23:51.622000+00:00
   [864691128455135296]
8.280265092849731
